# Diagnostic

This notebook tests the `diagnostic.py` module.

This module contains a set of methods that can be used to asses the convergence of the **underlying** sampler that provides the samples to FIGARO (e.g., LALInference or Bilby for skymaps).

## Angular coefficient

These functions computes the angular coefficient obtained from linear regression of a set of points.

### `angular_coefficient`


Given two arrays of points ($x$ and $f(x)$), returns the angular coefficient obtained assuming a linear relationship between $x$ and $f(x)$.

With just two points, this reduces to the usual formula for the angular coefficient of a straight line passing through two points:

In [ ]:
import numpy as np
from figaro.diagnostic import angular_coefficient

#(0,0) and (1,2)
print(angular_coefficient(x = np.array([0,1]), y = np.array([0,2])))

A set of points from a line:

In [ ]:
m = 4
q = 3

x = np.linspace(0,10, 3000)
y = m*x+q
print(angular_coefficient(x, y))

Let's add some Gaussian noise on the points:

In [ ]:
y = m*x+q + np.random.normal(loc = 0, scale = 2, size = len(x))
ac = angular_coefficient(x,y)
print(ac)

import matplotlib.pyplot as plt
t = plt.plot(x, y, lw = 0.5, color = 'steelblue', marker = '')
t = plt.plot(x, ac*x+q, color = 'r', ls = '--')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.grid(alpha = 0.5)

### `compute_angular_coefficients`

Given an array of points $x$, computes the angular coefficient as a function of the position on the array on all the adjacent windows of length $L$ that can be found in the array.

In [ ]:
from figaro.diagnostic import compute_angular_coefficients

L     = 100
n_pts = 1000

x = np.arange(n_pts) + np.random.normal(loc = 0, scale = 2, size = n_pts)
ac = compute_angular_coefficients(x, L = L)

plt.plot(np.arange(len(ac))+L, ac, lw = 0.8, ls = '--', color = 'steelblue')
plt.axhline(1, ls = '--', lw = 0.5, color = 'r')
plt.xlabel('$i$')
plt.ylabel('$m$')
plt.grid(alpha = 0.5)

### `plot_angular_coefficient`

The same plot as above can be obtained by calling `plot_angular_coefficient`:

In [ ]:
from figaro.diagnostic import plot_angular_coefficient

S = plot_angular_coefficient(x, L = L, ac_expected = 1, show = True, save = False)

## Entropy

This set of methods computes the differential entropy of a distribution, defined as\
$
S = -\int p(x) \log p(x) dx\,,
$\
via Monte Carlo integration. These methods work for any class with a `rvs` and `logpdf` method (e.g. `figaro.mixture.mixture/DPGMM/HDPGMM` or any `scipy.stats` class).

### `compute_entropy_single_draw`

This method computes the differential entropy of a single distribution (a FIGARO draw, for example) via Monte Carlo integration, expressed in bits.

We will test this method with a single Gaussian distribution. For this distribution, $S=\ln\Big(\sigma\sqrt{2\pi e}\Big)$

In [ ]:
from figaro.diagnostic import compute_entropy_single_draw
from scipy.stats import norm

mu = 0
sigma = 1
S_exp = np.log(sigma*np.sqrt(2*np.pi*np.e))

# In bits
S, dS = compute_entropy_single_draw(norm(loc = mu, scale = sigma), return_error = True)
# In nats
print('{0:.4f}+-{1:.4f}, expected: {2:.4f}'.format(S*np.log2(np.e), dS*np.log2(np.e), S_exp))

Uniform distribution, $S = \ln(x_{\mathrm{max}} - x_{\mathrm{min}})$:

In [ ]:
from scipy.stats import uniform

xmin = -5
xmax = 5
S_exp = np.log(xmax-xmin)
# Bits
S, dS = compute_entropy_single_draw(uniform(xmin, xmax-xmin), return_error = True)
# Nats
print('{0}+-{1}, expected: {2}'.format(S*np.log2(np.e), dS*np.log2(np.e), S_exp))

Exponential distribution, $S=1-\ln(\lambda)$:

In [ ]:
from scipy.stats import expon

l = 2
S_exp = 1 - np.log(l)
# Bits
S, dS = compute_entropy_single_draw(expon(scale = 1/l), return_error = True)
# Nats
print('{0:.4f}+-{1:.4f}, expected: {2:.4f}'.format(S*np.log2(np.e), dS*np.log2(np.e), S_exp))

### `compute_entropy`

Computes the entropy of an array of distributions (`for` loop wrapper):

In [ ]:
from figaro.diagnostic import compute_entropy

distributions = [norm(mu, sigma), uniform(xmin, xmax-xmin), expon(scale = 1/l)]
S, dS         = compute_entropy(distributions, n_draws = 1e4, return_error = True)
exp_entropies = [np.log(sigma*np.sqrt(2*np.pi*np.e)), np.log(xmax-xmin), 1 - np.log(l)]

for Si, dSi, S_exp, name in zip(S, dS, exp_entropies, ['Gaussian', 'Uniform', 'Exponential']):
    print('{0}: {1:.4f}+-{2:.4f}, expected {3:.4f}'.format(name, Si*np.log2(np.e), dSi*np.log2(np.e), S_exp))

This function is meant to study the evolution of the FIGARO reconstruction entropy as a function of the number of samples added to the mixture, as demonstrated in the following.

### `entropy`

This method calls `compute_entropy` on an array of distributions and makes a fancy plot with the entropy.

In the following, we will show the evolution of the FIGARO reconstruction entropy. The target distribution is a Gaussian distribution. 

In [ ]:
from figaro.mixture import DPGMM
from tqdm import tqdm

mu      = 0
sigma   = 1
n_samps = 10000

samples = norm(mu, sigma).rvs(n_samps)
mix     = DPGMM([[-5,5]])
draws   = []

for s in tqdm(samples):
    mix.add_new_point(s)
    draws.append(mix.build_mixture())

Now let's have a look at the entropy evolution:

In [ ]:
from figaro.diagnostic import entropy

S = entropy(draws,
            exp_entropy = np.log(sigma*np.sqrt(2*np.pi*np.e))/np.log2(np.e), # Entropy in bits
            show = True,
            save = False,
            )

## Autocorrelation


These functions compute the autocorrelation function for a set of functions, where the autocorrelation is defined as\
$
C(\tau) = \Big\langle \frac{\int(f_t(x) - \bar f(x))(f_{t+\tau}(x) - \bar f(x)) dx}{\int (f_t(x) - \bar f(x))^2 dx}\Big\rangle_t\,.
$

### `compute_autocorrelation`

This method computes the autocorrelation function.\
For a deterministic process in which we define $f_i(x) = y_i$ and $y_i = \lambda y_{i-1}$ with $\lambda < 1$, we have that $C(\tau) = \lambda^{\tau}$. 

In [ ]:
from figaro.diagnostic import compute_autocorrelation

ndata = 10000
acl = 0.9
y = np.zeros(ndata)
y[0] = 1

for i in range(0,len(w)-1):
    y[i+1] = y[i]*acl

draws = np.array([np.ones(10)*(y_i) for y_i in y])
tmax, autoc = compute_autocorrelation(draws, np.mean(draws, axis = 0), dx)

taus = np.arange(tmax)
plt.plot(taus, autoc, label = 'Autocorrelation')
plt.plot(taus, acl**taus, ls = '--', label = '$\lambda^{\\tau}$')
plt.legend(loc = 0, frameon = False)
plt.grid(alpha = 0.5)
l = plt.xlim(0,60)

### `autocorrelation`

This method calls the previous one to compute the autocorrelation and produces the relevant plot. It is tailored to work with FIGARO, therefore it expects a list of `mixture` instances.

In [ ]:
from figaro.diagnostic import autocorrelation
from figaro.mixture import DPGMM
from tqdm import tqdm

samples = norm().rvs(1000)
mix = DPGMM([[-5,5]])
n_draws = 100

draws = []
for _ in tqdm(range(n_draws)):
    draws.append(mix.density_from_samples(samples))
    
autocorr = autocorrelation(draws, save = False, show = True)
